In [1]:
# Basic imports
import os
import sys
import time
import timeit
import random
import pickle
import re
from itertools import product
import warnings

# System path modification
sys.path.insert(0, '..')

# Data handling
import pandas as pd
import numpy as np

# Machine learning imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer, MissingIndicator
from sklearn.model_selection import (
    train_test_split, KFold, StratifiedKFold, GroupKFold, StratifiedGroupKFold, LeaveOneOut, cross_validate, cross_val_score
)
from sklearn.linear_model import (
    LinearRegression, Lasso, LassoCV, MultiTaskLasso, MultiTaskLassoCV,
    ElasticNet, ElasticNetCV, MultiTaskElasticNet, MultiTaskElasticNetCV
)
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score

from sklearn.cross_decomposition import PLSRegression
from sklearn.inspection import permutation_importance

# Statistic imports 
from scipy.stats import ks_2samp
from scipy.special import kl_div
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import pdist, squareform

# Specialized imputation and visualization packages
import miceforest as mf
import missingno as msno
#from missforest import MissForest
#import magic
from src.gain import *

# Custom modules
from src.train import *
from src.functions import *
from src.plots import *
from src.dataset import *
from src.multixgboost import *
from src.wrapper import *
from src.debug import *

# Visualizatiokn 
import matplotlib.pyplot as plt
import seaborn as sns

# Deep learning and machine learning specific 
import torch
from pytorch_tabnet.tab_model import TabNetRegressor
import xgboost as xgb
import shap

from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.models import (
    GatedAdditiveTreeEnsembleConfig,
    DANetConfig,
    TabTransformerConfig,
    TabNetModelConfig,
)

# Ignore warnings
warnings.filterwarnings("ignore")

# Print CUDA availability for PyTorch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

from omegaconf import DictConfig
torch.serialization.safe_globals([DictConfig])

True
1


## Load data 

In [2]:
data = load_pickle_data_palettes()

results_pickle_folder = "../pickle/"

# Unpack data
df_X, df_y, df_all, df_FinalCombination = data["df_X"], data["df_y"], data["df_all"], data["df_FinalCombination"]
dict_select = data["dict_select"]

# Unpack colormaps
full_palette, gender_palette, dx_palette = data["colormaps"].values()

In [3]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

print(sum(idx_test))

print(df_all[idx_test].RID)

df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("category")

test_indices = [i for i, val in enumerate(idx_test) if val]

13
3609    2002
5631    4167
5662    4176
5780    4215
5950    4349
6069    4292
6077    4453
6085    4489
6224    4505
6400    4576
6429    4300
7021    2374
7192    4179
Name: RID, dtype: int64


# Leave-One-Complete-Out (LOCO-CV)

## All features

In [4]:
random_state=42

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("KNNImputer", KNNImputer(n_neighbors=1)),
]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("SimpleImputer_most_frequent", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.5079831261101071, 'learning_rate': 0.0769592094304232, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.8049983288913105})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [5]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=ordinal_features
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=False,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__

predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]

In [6]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: LinearRegression
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskElasticNet
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskElasticNet_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskLasso
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: MultiTaskLasso_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: RandomForestRegressor
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: XGBoostRegressor
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: XGBoostRegressor_tuned
Continuous Imputer: KNNImputer, Ordinal Imputer: SimpleImputer_most_frequent, Model: TabNetRegressor_default
Continuous Imputer: KNNImputer, Ordinal Imputer: S

In [7]:
# Initialize HDF5 file
results_file = '../pickle/training_3_loonona_dict_results.pickle'

In [8]:
if os.path.exists(results_file): 
    with open(results_file, "rb") as input_file:
        all_dict_results = pickle.load(input_file)

else : 
    all_dict_results = []

In [ ]:
if False : 
    params_comb = [{'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]},
    {'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}]

    for params in params_comb:
        all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"params": params})

Removed due to key-value match: params={'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'GatedAdditiveTreeEnsembleConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
Removed due to key-value match: params={'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'DANetConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
Removed due to key-value match: params={'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabTransformerConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
Removed due to key-value match: params={'ordinal_imputer': 'SimpleImputer_most_frequent', 'continuous_imputer': 'KNNImputer', 'model': 'TabNetModelConfig_tab', 'train_shape': [2893, 348], 'test_shape': [1, 348]}
Removed due to key-value match: params={'ordinal_imputer': 'NoImputer', 'continuous_imputer': 'NoImputer', 'model': 'GatedAdditiveTreeEnsembl

In [11]:
all_dict_results = clean_dict_list(all_dict_results, remove_if_none=False, remove_key_val={"fitting_time":None})

In [12]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : [df_X.shape[0]-1, df_X.shape[1]],
            "test_shape": [1, df_X.shape[1]]
        }
    
    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want

    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue

    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 
        print(test_nloc)

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X.loc[idx_train]
        df_X_test = df_X.loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )
            
            dict_results["imputation_time"].append(fold_dict_results["imputation_time"]) 
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])  

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'LinearRegression']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskLasso']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'RandomForestRegressor']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'XGBoostRegressor']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', 'KNNImputer', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['SimpleImputer_most_frequent', '

2025-06-13 02:11:08,839 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-06-13 02:11:08,855 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-06-13 02:11:08,861 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-06-13 02:22:32,886 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-06-13 02:22:32,886 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

Saving predictions in dict!


In [13]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
results_file

'../pickle/training_3_loonona_dict_results.pickle'

In [15]:
with open(results_file, "rb") as input_file:
    dict_results_loo_nona = pickle.load(input_file)

In [16]:
df_results_nona = pd.json_normalize(dict_results_loo_nona)

In [17]:
df_results_nona

,imputation_time,fitting_time,results_adj,results_org,params.ordinal_imputer,params.continuous_imputer,params.model,params.train_shape,params.test_shape,results_adj.mse_score,results_adj.mae_score,results_adj.r2,results_adj.explained_variance,results_adj.corr,results_org.mse_score,results_org.mae_score,results_org.r2,results_org.explained_variance,results_org.corr
0,"[3.3089418411254883, 3.2607581615448, 3.286860...","[0.11002111434936523, 0.24651741981506348, 0.1...",[{'y_pred': [[0.56338451 1.36697629 0.61842186...,[{'y_pred': [[0.75794363 1.56522677 0.49123517...,SimpleImputer_most_frequent,KNNImputer,LinearRegression,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[3.4814867973327637, 3.5424869060516357, 3.515...","[0.08670425415039062, 0.08908843994140625, 0.0...",[{'y_pred': [[0.10086018 0.10725904 0.05499171...,[{'y_pred': [[ 0.29541929 0.30550953 -0.07219...,SimpleImputer_most_frequent,KNNImputer,MultiTaskElasticNet,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[3.544888734817505, 3.5088696479797363, 3.4993...","[1.513960361480713, 1.5192315578460693, 1.5161...",[{'y_pred': [[0.43508893 1.25141221 0.45281623...,[{'y_pred': [[0.62964805 1.4496627 0.32562954...,SimpleImputer_most_frequent,KNNImputer,MultiTaskElasticNet_tuned,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[3.458850145339966, 3.4527065753936768, 3.4638...","[0.0770413875579834, 0.07488107681274414, 0.07...",[{'y_pred': [[-1.14021119e-10 -7.73912862e-10 ...,[{'y_pred': [[ 0.19455912 0.19825049 -0.12718...,SimpleImputer_most_frequent,KNNImputer,MultiTaskLasso,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[3.5399937629699707, 3.5302608013153076, 3.551...","[1.4914090633392334, 1.5125789642333984, 1.522...",[{'y_pred': [[0.50635131 1.33052966 0.55415854...,[{'y_pred': [[0.70091043 1.52878015 0.42697185...,SimpleImputer_most_frequent,KNNImputer,MultiTaskLasso_tuned,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"[3.5142085552215576, 3.3101305961608887, 3.446...","[52.41137504577637, 52.029266357421875, 51.895...",[{'y_pred': [[0.29340068 0.37610565 0.22158855...,[{'y_pred': [[0.48795979 0.57435613 0.09440186...,SimpleImputer_most_frequent,KNNImputer,RandomForestRegressor,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,[],[],[],[],SimpleImputer_most_frequent,KNNImputer,XGBoostRegressor,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[],[],[],[],SimpleImputer_most_frequent,KNNImputer,XGBoostRegressor_tuned,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"[3.467628240585327, 3.957735300064087, 3.52839...","[13.44585394859314, 14.944092273712158, 14.055...",[{'y_pred': [[-0.33021685 0.02510688 0.12306...,[{'y_pred': [[-0.13565774 0.22335736 -0.00412...,SimpleImputer_most_frequent,KNNImputer,TabNetRegressor_default,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"[3.212691068649292, 3.3576810359954834, 3.2116...","[12.390780925750732, 13.0169358253479, 12.1248...",[{'y_pred': [[0.38124186 0.36439073 0.21427807...,[{'y_pred': [[0.57580098 0.56264122 0.08709138...,SimpleImputer_most_frequent,KNNImputer,TabNetRegressor_custom,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Train only on MRI features

In [18]:
idx_train = list(df_X.isna().any(axis=1))
idx_test = list(~df_X.isna().any(axis=1))

set_intersect_rid = set(df_all[idx_train].RID).intersection(set(df_all[idx_test].RID))
intersect_rid_idx = df_all.RID.isin(set_intersect_rid)

for i, bool_test in enumerate(idx_test): 
    if intersect_rid_idx.iloc[i] & bool_test:
        idx_test[i] = False
        idx_train[i] = True

In [19]:
df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]] = df_X[["APOE_epsilon2", "APOE_epsilon3", "APOE_epsilon4"]].astype("category")

In [20]:
df_X_train = df_X[dict_select["MRIth"]].loc[idx_train]
df_X_test = df_X[dict_select["MRIth"]].loc[idx_test]

df_y_train = df_y.loc[idx_train]
df_y_test = df_y.loc[idx_test]

c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

In [21]:
random_state=42
n_imputation_iter = 10

# Continuous Imputer List (list of tuples with unique strings and corresponding instances)
continuous_imputer_list = [
    ("NoImputer", KNNImputer(n_neighbors=1)),

]

# Ordinal Imputer List (list of tuples with unique strings and corresponding instances)
ordinal_imputer_list = [
    ("NoImputer", SimpleImputer(strategy="most_frequent")),
]

# Predictive Models List (list of tuples with unique strings and corresponding instances)
predictive_models_list = [
    ("LinearRegression", LinearRegression()),
    ("MultiTaskElasticNet", MultiTaskElasticNet()),
    ("MultiTaskElasticNet_tuned", MultiTaskElasticNet(**{'alpha': 0.01, 'l1_ratio': 0.01})),
    ("MultiTaskLasso", MultiTaskLasso()),
    ("MultiTaskLasso_tuned", MultiTaskLasso(**{'alpha': 0.001})),
    ("RandomForestRegressor", RandomForestRegressor()),
    ("XGBoostRegressor", XGBoostRegressor()),
    ("XGBoostRegressor_tuned", XGBoostRegressor(**{'colsample_bytree': 0.5079831261101071, 'learning_rate': 0.0769592094304232, 'max_depth': 6, 'min_child_weight': 7, 'subsample': 0.8049983288913105})),
    ("TabNetRegressor_default", TabNetModelWrapper(n_a=8, n_d=8)),
    ("TabNetRegressor_custom", TabNetModelWrapper(n_a=32, n_d=32)),
    ("PLSRegression_4_components", PLSRegression(n_components=4))
]

In [22]:
ordinal_features = ['APOE_epsilon2', 'APOE_epsilon3', 'APOE_epsilon4']
continuous_features = [col for col in df_X_train.columns if col not in ordinal_features]

# Prepare Tabular configurations (shared for all PyTorch models)
data_config = DataConfig(
    target=df_y_train.columns.tolist(),
    continuous_cols=continuous_features,
    categorical_cols=[]
)
trainer_config = TrainerConfig(
    batch_size=1024, max_epochs=10, auto_lr_find=True,
    early_stopping="valid_loss", early_stopping_mode="min", early_stopping_patience=5,
    checkpoints="valid_loss", load_best=True, progress_bar="nones",
)
optimizer_config = OptimizerConfig()
head_config = LinearHeadConfig(dropout=0.1).__dict__
predictive_models_list += [
    ("GatedAdditiveTreeEnsembleConfig_tab", 
    TabularModelWrapper(
        GatedAdditiveTreeEnsembleConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        gflu_stages=6,
        gflu_dropout=0.0,
        tree_depth=5,
        num_trees=20,
        chain_trees=False,
        share_head_weights=True), data_config, trainer_config, optimizer_config 
    )),
    ("DANetConfig_tab",
    TabularModelWrapper(
        DANetConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_layers=8,
        k=5,
        dropout_rate=0.1), data_config, trainer_config, optimizer_config
    )),
    ("TabTransformerConfig_tab",
        TabularModelWrapper(
        TabTransformerConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        embedding_initialization="kaiming_uniform",
        embedding_bias=False), data_config, trainer_config, optimizer_config
    )),
    ("TabNetModelConfig_tab",
        TabularModelWrapper(
        TabNetModelConfig(
        task="regression",
        head="LinearHead",
        head_config=head_config,
        n_d=8,
        n_a=8,
        n_steps=3,
        gamma=1.3,
        n_independent=2,
        n_shared=2), data_config, trainer_config, optimizer_config
    )),
]


In [23]:
# Generate all combinations
combinations = list(product(continuous_imputer_list, ordinal_imputer_list, predictive_models_list))

# Display all combinations
for continuous_imputer, ordinal_imputer, model in combinations:
    print(f"Continuous Imputer: {continuous_imputer[0]}, Ordinal Imputer: {ordinal_imputer[0]}, Model: {model[0]}")

print(f"Combinations of preprocessing and models to test : {len(combinations)}")

Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: LinearRegression
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskElasticNet_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: MultiTaskLasso_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: RandomForestRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: XGBoostRegressor_tuned
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_default
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: TabNetRegressor_custom
Continuous Imputer: NoImputer, Ordinal Imputer: NoImputer, Model: PLSRegression_4_components
Continuous Imputer: NoImputer, Ordinal 

In [24]:
# Initialize HDF5 file
results_file = '../pickle/training_3_loonona_dict_results.pickle'

with open('../pickle/training_3_loonona_dict_results.pickle', "rb") as input_file:
    all_dict_results = pickle.load(input_file)

In [25]:
for continuous_imputer, ordinal_imputer, model in combinations:
    name_continuous_imputer, continuous_imputer_instance = continuous_imputer
    name_ordinal_imputer, ordinal_imputer_instance = ordinal_imputer
    name_model, model_instance = model

    params = {
            "ordinal_imputer": name_ordinal_imputer, 
            "continuous_imputer": name_continuous_imputer, 
            "model": name_model, "train_shape" : [df_X.shape[0]-1, df_X.shape[1]],
            "test_shape": [1, df_X.shape[1]]
        }
    
    # Define the subset of keys you care about
    keys_to_check = ['ordinal_imputer', 'continuous_imputer', 'model']  # or whatever subset you want

    # Check if a result in all_dict_results has the same values for just those keys
    if any(all(result['params'].get(k) == params.get(k) for k in keys_to_check) for result in all_dict_results):
        print(f"Skipping existing combination (subset match): {[params[k] for k in keys_to_check]}")
        continue

    dict_results = {
            "params": params, 
            "imputation_time": [],
            "fitting_time": [], 
            "results_adj": [], 
            "results_org": []
        }

    for test_nloc in test_indices: 
        print(test_nloc)

        idx_train = [True for i in range(df_X.shape[0])]
        idx_test = [False for i in range(df_X.shape[0])]

        idx_test[test_nloc] = True
        idx_train[test_nloc] = False

        df_X_train = df_X.loc[idx_train]
        df_X_test = df_X.loc[idx_test]

        df_y_train = df_y.loc[idx_train]
        df_y_test = df_y.loc[idx_test]

        c_train = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_train]
        c_test = df_all[["AGE", "PTGENDER", "PTEDUCAT"]].iloc[idx_test]

        try: 
        
            # Now you can call your `train_model` function with these components
            fold_dict_results = train_imputer_model(
                df_X_train, df_X_test, df_y_train, df_y_test,
                c_train, c_test,
                ordinal_imputer_instance, name_ordinal_imputer,
                continuous_imputer_instance, name_continuous_imputer,
                model_instance, name_model,
                separate_imputers=True  # Or however you want to specify
            )
            
            dict_results["imputation_time"].append(fold_dict_results["imputation_time"]) 
            dict_results["fitting_time"].append(fold_dict_results["fitting_time"])  
            dict_results["results_adj"].append(fold_dict_results["results_adj"])  
            dict_results["results_org"].append(fold_dict_results["results_org"])  

        except Exception as e:  

            print(e)
            
    # Optionally keep the all_dict_results list updated
    all_dict_results.append(dict_results)

    # Save the updated results back to the pickle file
    with open(results_file, 'wb') as f:
        pickle.dump(all_dict_results, f)


Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'LinearRegression']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskElasticNet']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskElasticNet_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskLasso']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'MultiTaskLasso_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'RandomForestRegressor']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'XGBoostRegressor']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'XGBoostRegressor_tuned']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'TabNetRegressor_default']
Skipping existing combination (subset match): ['NoImputer', 'NoImputer', 'TabNetRegressor_custom']
Skipping existing combination (s

2025-06-13 02:22:40,678 - {pytorch_tabular.tabular_model:146} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-06-13 02:22:40,690 - {pytorch_tabular.tabular_model:548} - INFO - Preparing the DataLoaders

2025-06-13 02:22:40,695 - {pytorch_tabular.tabular_datamodule:522} - INFO - Setting up the datamodule for          
regression task

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

2025-06-13 02:37:00,916 - {pytorch_tabular.tabular_model:689} - INFO - Training the model completed

2025-06-13 02:37:00,917 - {pytorch_tabular.tabular_model:1529} - INFO - Loading the best model

Saving predictions in dict!


In [26]:
# Store data (serialize)
with open(results_file, 'wb') as handle:
    pickle.dump(all_dict_results, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
with open(results_file, "rb") as input_file:
    all_dict_results = pickle.load(input_file)

## Print Table for paper

In [28]:
with open('../pickle/training_3_loonona_dict_results.pickle', "rb") as input_file:
    dict_results_loo_nona = pickle.load(input_file)

In [29]:
df_results_nona = pd.json_normalize(dict_results_loo_nona)
df_results_nona

,imputation_time,fitting_time,results_adj,results_org,params.ordinal_imputer,params.continuous_imputer,params.model,params.train_shape,params.test_shape,results_adj.mse_score,results_adj.mae_score,results_adj.r2,results_adj.explained_variance,results_adj.corr,results_org.mse_score,results_org.mae_score,results_org.r2,results_org.explained_variance,results_org.corr
0,"[3.3089418411254883, 3.2607581615448, 3.286860...","[0.11002111434936523, 0.24651741981506348, 0.1...",[{'y_pred': [[0.56338451 1.36697629 0.61842186...,[{'y_pred': [[0.75794363 1.56522677 0.49123517...,SimpleImputer_most_frequent,KNNImputer,LinearRegression,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[3.4814867973327637, 3.5424869060516357, 3.515...","[0.08670425415039062, 0.08908843994140625, 0.0...",[{'y_pred': [[0.10086018 0.10725904 0.05499171...,[{'y_pred': [[ 0.29541929 0.30550953 -0.07219...,SimpleImputer_most_frequent,KNNImputer,MultiTaskElasticNet,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[3.544888734817505, 3.5088696479797363, 3.4993...","[1.513960361480713, 1.5192315578460693, 1.5161...",[{'y_pred': [[0.43508893 1.25141221 0.45281623...,[{'y_pred': [[0.62964805 1.4496627 0.32562954...,SimpleImputer_most_frequent,KNNImputer,MultiTaskElasticNet_tuned,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[3.458850145339966, 3.4527065753936768, 3.4638...","[0.0770413875579834, 0.07488107681274414, 0.07...",[{'y_pred': [[-1.14021119e-10 -7.73912862e-10 ...,[{'y_pred': [[ 0.19455912 0.19825049 -0.12718...,SimpleImputer_most_frequent,KNNImputer,MultiTaskLasso,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[3.5399937629699707, 3.5302608013153076, 3.551...","[1.4914090633392334, 1.5125789642333984, 1.522...",[{'y_pred': [[0.50635131 1.33052966 0.55415854...,[{'y_pred': [[0.70091043 1.52878015 0.42697185...,SimpleImputer_most_frequent,KNNImputer,MultiTaskLasso_tuned,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"[3.5142085552215576, 3.3101305961608887, 3.446...","[52.41137504577637, 52.029266357421875, 51.895...",[{'y_pred': [[0.29340068 0.37610565 0.22158855...,[{'y_pred': [[0.48795979 0.57435613 0.09440186...,SimpleImputer_most_frequent,KNNImputer,RandomForestRegressor,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,[],[],[],[],SimpleImputer_most_frequent,KNNImputer,XGBoostRegressor,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,[],[],[],[],SimpleImputer_most_frequent,KNNImputer,XGBoostRegressor_tuned,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"[3.467628240585327, 3.957735300064087, 3.52839...","[13.44585394859314, 14.944092273712158, 14.055...",[{'y_pred': [[-0.33021685 0.02510688 0.12306...,[{'y_pred': [[-0.13565774 0.22335736 -0.00412...,SimpleImputer_most_frequent,KNNImputer,TabNetRegressor_default,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"[3.212691068649292, 3.3576810359954834, 3.2116...","[12.390780925750732, 13.0169358253479, 12.1248...",[{'y_pred': [[0.38124186 0.36439073 0.21427807...,[{'y_pred': [[0.57580098 0.56264122 0.08709138...,SimpleImputer_most_frequent,KNNImputer,TabNetRegressor_custom,"[2893, 348]","[1, 348]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def generate_metric_table(
    results_list,
    targets,
    metric_name,
    source="Adjusted",
    float_format="%.3f",
    csv_filename=None,
    sort_order="ascending"  # or "descending"
):
    """
    Create a LaTeX table for a single metric across targets, models, and imputers.
    Optionally export the same table as CSV and sort by mean performance.

    Parameters
    ----------
    results_list : list of dict
        List of experiment results.
    targets : list of str
        Target names (e.g., ['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN']).
    metric_name : str
        Metric to extract (e.g., 'mae_score').
    source : str
        'Adjusted' or 'Original'.
    float_format : str
        Format for floats (e.g., '%.3f').
    csv_filename : str or None
        If provided, saves the table to CSV.
    sort_order : str
        'ascending' or 'descending' for sorting by mean.

    Returns
    -------
    str
        LaTeX-formatted table string.
    """
    rows = []
    version_key = "results_adj" if source.lower() == "adjusted" else "results_org"

    for res in results_list:
        result_block = res.get(version_key)
        if result_block is None:
            continue

        metric_values = result_block.get(metric_name)
        if metric_values is None:
            continue

        if len(metric_values) != len(targets):
            continue

        ordinal_imputer = res["params"].get("ordinal_imputer")
        model = res["params"].get("model")

        values = np.array(metric_values, dtype=np.float64)
        mean_val = np.mean(values)
        std_val = np.std(values)

        row = {
            "Ordinal Imputer": ordinal_imputer,
            "Model": model,
            "Mean": mean_val,  # for sorting
            "Mean ± SD": f"{mean_val:.3f} ± {std_val:.3f}",
        }
        row.update({target: val for target, val in zip(targets, values)})
        rows.append(row)

    df = pd.DataFrame(rows)

    # Reorder columns for display
    display_cols = ["Ordinal Imputer", "Model"] + targets + ["Mean ± SD"]
    df = df.sort_values(by="Mean", ascending=(sort_order == "ascending"))
    df = df[display_cols]

    # Save CSV
    if csv_filename:
        df.to_csv(csv_filename, index=False)

    # LaTeX output
    latex_table = df.to_latex(
        index=False,
        escape=False,
        float_format=float_format,
        caption=f"{metric_name.replace('_', ' ').upper()} across targets",
        label=f"tab:{metric_name}",
        longtable=False
    )

    return df, latex_table


In [31]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='corr',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_corr_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_mae)

AttributeError: 'list' object has no attribute 'get'

In [ ]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='r2',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_r2_adjusted_sorted.csv",
    sort_order="descending"
)
print(latex_mae)

In [ ]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mse_score',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_mse_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

In [ ]:
latex_df, latex_mae = generate_metric_table(
    results_list=all_dict_results,
    targets=['ADNI_MEM', 'ADNI_EF', 'ADNI_VS', 'ADNI_LAN'],
    metric_name='mae_score',
    source="Adjusted",
    csv_filename="../tables/2_training_train_test_mae_adjusted_sorted.csv",
    sort_order="ascending"
)
print(latex_mae)

\begin{table}
\caption{MAE SCORE across targets}
\label{tab:mae_score}
\begin{tabular}{llrrrrl}
\toprule
Ordinal Imputer & Model & ADNI_MEM & ADNI_EF & ADNI_VS & ADNI_LAN & Mean ± SD \\
\midrule
SimpleImputer_most_frequent & MultiTaskLasso_tuned & 0.622 & 0.551 & 0.578 & 0.595 & 0.586 ± 0.026 \\
SimpleImputer_most_frequent & LinearRegression & 0.640 & 0.551 & 0.565 & 0.600 & 0.589 ± 0.034 \\
SimpleImputer_most_frequent & MultiTaskElasticNet_tuned & 0.630 & 0.554 & 0.580 & 0.600 & 0.591 ± 0.028 \\
SimpleImputer_most_frequent & TabNetRegressor_default & 0.691 & 0.608 & 0.536 & 0.595 & 0.607 ± 0.055 \\
SimpleImputer_most_frequent & RandomForestRegressor & 0.669 & 0.640 & 0.608 & 0.648 & 0.641 ± 0.022 \\
NoImputer & XGBoostRegressor & 0.754 & 0.635 & 0.643 & 0.644 & 0.669 ± 0.049 \\
NoImputer & XGBoostRegressor_tuned & 0.735 & 0.715 & 0.630 & 0.613 & 0.673 ± 0.052 \\
NoImputer & RandomForestRegressor & 0.755 & 0.684 & 0.635 & 0.632 & 0.677 ± 0.050 \\
NoImputer & MultiTaskLasso_tuned & 0.72

In [ ]:
latex_df.Model.value_counts()

Model
MultiTaskLasso_tuned          2
LinearRegression              2
MultiTaskElasticNet_tuned     2
TabNetRegressor_default       2
RandomForestRegressor         2
PLSRegression_4_components    2
TabNetRegressor_custom        2
MultiTaskElasticNet           2
MultiTaskLasso                2
XGBoostRegressor_tuned        1
XGBoostRegressor              1
Name: count, dtype: int64